In [1]:
import pydss
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
output_notebook()

sqrt2 = np.sqrt(2.0)
sqrt3 = np.sqrt(3.0)
pi = np.pi

Loading BokehJS ...

## Average model 1

In [110]:



E = 500.0/sqrt3 # phase voltage rms
f_e = 60.0
R_th = 0.01
L_th = 500.0e-6
R_ac = 0.091
L_ac = 9.545e-3
R_dc = 0.0
L_dc = 0.0
C = 500e-6

R_load = 0.1


L_s = L_th+L_ac
omega_e = 2*np.pi*f_e
R_s = R_th+R_ac

i_dc_av = 16.78
v_c = 588.0
v_dc = 400.0
Dt = 100.0e-6
t = 0.0
T = []
V_c = []
I_dc = []
for it in range(3000):
    
    t += Dt
    #if t>0.1: v_dc = 1000.0
    #if t>0.2: v_dc = 500.0
        
    T += [t]
    V_c += [v_c]
    I_dc += [i_dc_av]
    
    dv_c = 1.0/C*(i_dc_av - (v_c-v_dc)/R_load)
    v_d = 3*sqrt3/pi*sqrt2*E - (3/pi*omega_e*L_s+R_dc)*i_dc_av
    v_d_p = np.maximum(v_d,v_dc)
    di_dc_av = (v_d_p  - v_c)/(2*L_s+L_dc)
    mu_arg = (1-np.sqrt(2.0/3.0)*omega_e*L_s/(E)*i_dc_av)
    if mu_arg > 1.0: 
        mu_arg = 1.0
        mu = 0.0
       
    else: 
        mu = np.arccos(mu_arg)
    
    v_c += Dt*dv_c
    i_dc_av += Dt*di_dc_av

#if i_dc_av < 0.1: i_dc_av = 10
i_qs_com = -2.0*sqrt3/pi*i_dc_av*( np.sin(mu-5*pi/6) + np.sin(5*pi/6)) - 3.0/pi*sqrt2*E/(omega_e*L_s)*(np.cos(mu)-1) +3/(4*pi)*sqrt2*E/(omega_e*L_s)*(np.cos(2*mu)-1)                             
i_ds_com = -2.0*sqrt3/pi*i_dc_av*( np.cos(5*pi/6) - np.cos(mu-5*pi/6)) - 3.0/pi*sqrt2*E/(omega_e*L_s)*(np.sin(mu)) +3/(4*pi)*sqrt2*E/(omega_e*L_s)*(np.sin(2.0*mu) + 2*mu)                             
i_qs_cond = -2.0*sqrt3/pi*i_dc_av*( np.sin(7*pi/6) - np.sin(mu + 5*pi/6))
i_ds_cond = -2.0*sqrt3/pi*i_dc_av*( np.cos(mu + 5*pi/6) - np.cos(7*pi/6))

i_qs = i_qs_com + i_qs_cond
i_ds = i_ds_com + i_ds_cond
print(mu)
print(v_c)
print(i_dc_av)
print(i_qs_com)
print(i_ds_com)
print(i_qs_cond)
print(i_ds_cond)

i_ds = i_ds_com + i_ds_cond
i_qs = i_qs_com + i_qs_cond


print(i_ds)
print(i_qs)

1.36145331802
448.687912113
74.1678623012
69.1882078964
75.9473902902
-19.8900950213
-16.1082704718
59.8391198184
49.2981128751


In [111]:
p = figure(width=600, height=200)
lin = p.line(T,V_c, color="red", alpha=0.5, line_width=2)
p.add_tools(HoverTool())
show(p)


In [112]:
p = figure(width=600, height=200)
lin = p.line(T,I_dc, color="red", alpha=0.5, line_width=2)
p.add_tools(HoverTool())
show(p)

In [114]:
import electric
obj_rect = electric.rectifier_3ph()

In [115]:
import numba
V_peak = E*np.sqrt(2.0)

@numba.jit(nopython=True)
def run_eval():
    N_steps = int(0.1e6)
    N_states = 3
    x = np.zeros((N_states,1))
    f = np.zeros((N_states,1))
    X = np.zeros((N_states,N_steps))
    V = np.zeros((3,1),dtype=np.complex128)
    out = np.zeros((6,N_steps))
    #self.out_cplx = np.zeros((72,N_steps),np.complex128)
        
    T = np.zeros((N_steps,1))
    
    obj_src = electric.sourceNph()
    obj_rect = electric.rectifier_3ph()
    
    obj_src.omega = omega_e
    obj_rect.v_dc_src = v_dc
    obj_rect.R_dc = R_load
    obj_rect.L = L_s
    obj_rect.R = R_s
    obj_rect.C = C

    V[0] = V_peak*np.exp(1j*np.deg2rad(0.0))   
    V[1] = V_peak*np.exp(1j*np.deg2rad(-120.0))   
    V[2] = V_peak*np.exp(1j*np.deg2rad(-240.0))   
    
    obj_src.vec = V
    
    dt = 10.0e-6
    t = 0.0
    X[:,0] = x[:,0]
    T[0,0] = t
    for it in range(1,N_steps):  
            t += dt
        
            obj_rect.x = x
            obj_rect.v_s_abc = obj_src.h_eval(t)
 
          
            f[0:4] = obj_rect.f_eval(t) 

            x = x + dt*f
            
            X[:,it] = x[:,0]
            out[0:3,it] =obj_rect.i_abc[:,0]
            out[3:6,it] =obj_rect.v_s_abc[:,0]
            T[it,0] = t
            
    return T,X,out

In [116]:
%timeit run_eval()

The slowest run took 4.06 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 227 ms per loop


In [117]:
T,X,out = run_eval()

In [118]:
fig = figure(width=600, height=300)
decim = 10
fig.line(T[0:-1:decim,0],out[3,0:-1:decim], line_width=1, color='red')
fig.line(T[0:-1:decim,0],out[4,0:-1:decim], line_width=1, color='green')    
fig.line(T[0:-1:decim,0],out[5,0:-1:decim], line_width=1, color='blue')  
show(fig)

fig = figure(width=600, height=300)
decim = 10
fig.line(T[0:-1:decim,0],out[0,0:-1:decim], line_width=1, color='red')
fig.line(T[0:-1:decim,0],out[1,0:-1:decim], line_width=1, color='green')    
fig.line(T[0:-1:decim,0],out[2,0:-1:decim], line_width=1, color='blue')  
show(fig)

fig = figure(width=600, height=300)
decim = 10
fig.line(T[0:-1:decim,0],X[2,0:-1:decim], line_width=1, color='red')
show(fig)

In [127]:
import ctrl

@numba.jit(nopython=True)
def park_full(T,abc):
    dq0 = np.copy(abc)
    for it in range(T.shape[0]-1):
        dq0[0:3,it] = ctrl.park(abc[0:3,it],omega_e*T[it,0])
    return dq0
        
        

In [128]:
%timeit park_full(T,out)

1 loop, best of 3: 170 ms per loop


In [129]:
I_dq0 = park_full(T,out)

In [130]:
fig = figure(width=600, height=300)
decim = 10
fig.line(T[0:-1:decim,0],I_dq0[0,0:-1:decim], line_width=1, color='red')
fig.line(T[0:-1:decim,0],I_dq0[1,0:-1:decim], line_width=1, color='green')    
fig.line(T[0:-1:decim,0],I_dq0[2,0:-1:decim], line_width=1, color='blue')  
fig.line([T[0,0],T[-1,0]],[i_ds,i_ds], line_width=1, color='blue') 
fig.line([T[0,0],T[-1,0]],[i_qs,i_qs], line_width=1, color='blue') 
show(fig)